In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("../data/mva2023_sod4bird_train/images/00001.jpg", cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
import torch

In [ ]:
data = torch.tensor(img).unsqueeze(0).float()
data.shape

In [ ]:
data = data.permute(0, 3, 1, 2)
data.shape

In [ ]:
import Main.swin_transformer as st
from importlib import reload
reload(st)
from Main.swin_transformer import SwinTransformer

In [ ]:
model = SwinTransformer(
    img_size=2016,
    patch_size=4,
    in_chans=3,
    num_classes=1,
    embed_dim=96,
    depths=[2, 2, 6, 2],
    num_heads=[3, 6, 12, 24],
    window_size=7,
    mlp_ratio=4.0,
    qkv_bias=True,
    qk_scale=None,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    drop_path_rate=0.0,
    norm_layer=torch.nn.LayerNorm,
    ape=False,
    patch_norm=True,
    use_checkpoint=False,
    fused_window_process=False,
)
model

In [ ]:
data.shape

In [ ]:
img = cv2.resize(img, (2016, 2016))
data = torch.tensor(img).unsqueeze(0).float()
data = data.permute(0, 3, 1, 2)
data.shape

In [ ]:
model(data).shape

In [ ]:
import swin_T as swin
reload(swin)
from swin_T import swin_t

In [ ]:
model = swin_t()
model

In [ ]:
pred = model(data)
pred.shape

In [ ]:
from neck import Neck
from backbone import Backbone

import torch
import torch.nn as nn

In [ ]:
temp = torch.randn(1, 3, 1024, 1024)

In [ ]:
backbone_model = Backbone(hid_dim=96, layers=(2, 2, 2, 2), heads=(3, 6, 12, 24))

In [ ]:
backbone_out, feature_maps = backbone_model(temp)

In [ ]:
backbone_out.shape

In [ ]:
neck_model = Neck(hid_dim=96, layers=(2, 2, 2, 2), heads= (3, 6, 12, 64), channels=768, feature_maps=feature_maps)

In [ ]:
for i in range(4):
    print(feature_maps[i].shape)

In [ ]:
neck_out = neck_model(backbone_out.permute(0, 2, 3, 1))

In [ ]:
neck_out.shape